In [ ]:
# default_exp input_file

# Input file

> Convenience function to construct the input file required by py2gift.

In [ ]:
# export

import pathlib
from typing import Union, Optional, Callable

In [ ]:
test_file = '_input_.yaml'

# Header

A function to write the header of the file.
* `file`: name of the input file to be created

In [ ]:
# export

def write_header(file: Union[str, pathlib.Path], output_file: str, pictures_directory: str, ) -> None:
    
    with open(file, 'w') as f:
        
        f.write(f'output file: {output_file}\n')
        f.write(f'pictures base directory: {pictures_directory}\n')
        f.write("path to gift-wrapper: '~/gift-wrapper/wrap.py'\n")
        f.write('\n')
        
        f.write('categories:\n')

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')

In [ ]:
%cat {test_file}

# Class

## write_class_preamble

A function to write the preamble for a class which includes the name of the category. Strictly speaking, it's not necessary to create a new category every time a class is added, but this is more general and a category can show up many times in the input file.
* `file`: name of the input file to be *appended to*
* `category_name`: name of the category in which the class will be encompassed
* `base_category`: a parent category for `category_name` (optional)
* `test_mode`: if `True`, the last two parameters are overriden and the category is simply called `test`

For convenience, the final `category_name` (notice that it might get tweaked because of the other parameters) is returned.

In [ ]:
# export

def write_class_preamble(file: Union[str, pathlib.Path], category_name: str, base_category: Optional[str] = None, test_mode: bool = False) -> Union[str, list]:

    # file is *appended* (not overwritten)
    with open(file, 'a') as f:

        f.write('\n')
        
        if test_mode:
            
            category_name = 'test'
        
        else:
        
            if base_category:

                category_name = [base_category, f'{base_category}/{category_name}']
        
        f.write(f'  - name: {category_name}\n\n')
        f.write('    classes:\n')
    
    return category_name

In [ ]:
write_class_preamble(test_file, 'Test category')

In [ ]:
%cat {test_file}

Parameter `base_category` allows to create hierarchical categories

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')
write_class_preamble(test_file, 'Test category', base_category='base')

In [ ]:
%cat {test_file}

In `test_mode`

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')
write_class_preamble(test_file, 'Test category', base_category='base', test_mode=True)

In [ ]:
%cat {test_file}

## write_class_closing

A function to write the closing settings of a class.
* `file`: name of the input file to be *appended to*
* `n_instances`: number of questions of this class that will be created (the `setup` method of the class should be non-deterministic)
* `time`: time in minutes (an *integer*) estimated necessary to solve the question

In [ ]:
# export

def write_class_closing(file: Union[str, pathlib.Path], n_instances: int, time: Optional[int] = None) -> None:

    with open(file, 'a') as f:

        f.write(f'\n        number of instances: {n_instances}')
        
        if time:
            
            f.write(f'\n        time: {time}')

In [ ]:
write_class_closing(test_file, 2, 15)

In [ ]:
%cat {test_file}

In [ ]:
!rm {test_file}

# Categories

In [ ]:
# export

def function_to_make_hierarchical_category_name(base_category: str) -> Callable[[str], list]:
    
    def make_subcategory(category: str) -> list:
    
        return [base_category, f'{base_category}/{category}']
    
    return make_subcategory

In [ ]:
f = function_to_make_hierarchical_category_name('2020 exam')
f('Random')